# Guardrails

Un guardrail es una validación que se coloca alrededor del input o output del modelo, con el objetivo de asegurar que el output del modelo cumple con las expectativas y politicas de seguridad.

En los chatbots encontramos problemas constantemente:

- Alucinaciones
- Que el cliente muestre información sensible que no debe compartirse con los proveedores de LLMs
- Que el chtabot conteste preguntas no relacionadas con el negocio
- Que el chatbot hable sobre la competencia



Ejemplo Input guatdrails validan:

- Intento de jailbraking
- La pregunta del usuario tiene que ver con el objetivo del chatbot?
- El cliente está introduciendo información sensible?

eiemplo Output guardrails validan

- El LLM está compartiendo información sensible
- Si la respuesta se basa en la base de conocimiento

In [15]:
# # Warning control
# import warnings
# warnings.filterwarnings('ignore')

# Load environment variables
from dotenv import load_dotenv

load_dotenv()

import os
import yaml
from crewai import Agent, Task, Crew

In [16]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

In [17]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Ejemplo de Guardrail

Este es un ejemplo muy basico pero que muestra perfectamente como funiona un Guardrail. Vamos a construir un Agente, que escribe post para redes sociales. Y vamos a construir un Guardrail, que verifique que el post tiene menos de 1000 caracteres. 

Si el post supera esos caracteres, el guardrail enviara un error al agente, para que este vuelva a iterar, hasta crear un post de menos de 1000 caracteres.

In [18]:
from typing import Tuple, Any

def validate_post_length(task_output: Any) -> Tuple[bool, Any]:
    """
    Validates that the post content does not exceed 1000 characters.
    
    Args:
        task_output: The TaskOutput object to validate
        
    Returns:
        Tuple[bool, Any]: 
        - If valid: (True, validated_content)
        - If invalid: (False, error_message)
    """
    try:
        # Acceder al contenido de la respuesta final
        result = task_output.result
        
        if isinstance(result, str):
            char_count = len(result)
            
            if char_count > 1000:
                return (False, f"Post content exceeds maximum length of 1000 characters. Current length: {char_count}")

            return (True, result)
        else:
            return (False, f"Invalid result type. Expected string, got {type(result)}")
        
    except AttributeError:
        # Si no podemos acceder a .result, intentamos convertir directamente el task_output a string
        try:
            result = str(task_output)
            char_count = len(result)
            
            if char_count > 1000:
                return (False, f"Post content exceeds maximum length of 1000 characters. Current length: {char_count}")

            return (True, result)
        except Exception as e:
            return (False, f"Could not process content: {str(e)}")
        
    except Exception as e:
        return (False, f"Unexpected error during length validation: {str(e)}")

In [19]:
# Creating Agents
content_creator = Agent(
  config=agents_config['content_creator']
)

# Creating Tasks
post_creation = Task(
  config=tasks_config['post_creation'],
  agent=content_creator,
  guardrail=validate_post_length
)

# Creating Crew
crew = Crew(
  agents=[
    content_creator,
  ],
  tasks=[
    post_creation,
  ],
  verbose=True
)

Overriding of current TracerProvider is not allowed


In [20]:


input = {'topic':'Benefits of chatbots for customer support'}

In [21]:

# Run the crew
result = crew.kickoff(
  inputs=input
)

# Agent: Experto en escribir posts
## Task: Escribe un post sobre Benefits of chatbots for customer support de unas 150 palabras, con tono simple para que cualquier publico pueda entender.



# Agent: Experto en escribir posts
## Final Answer: 
¡Hola! Hoy vamos a hablar sobre los beneficios de los chatbots para el soporte al cliente. ¿Alguna vez has tenido una pregunta rápida y no sabes a quién acudir? Aquí es donde entran los chatbots. Estos pequeños ayudantes automáticos están disponibles 24/7, lo que significa que puedes obtener respuestas a cualquier hora del día, incluso a las 3 a.m. 

Además, los chatbots son rápidos. Pueden manejar múltiples preguntas al mismo tiempo, así que no hace falta esperar en una larga fila telefónica. Esto no solo mejora tu experiencia, sino que también ahorra tiempo. 

Otra gran ventaja es que los chatbots pueden responder preguntas frecuentes y resolver problemas simples, dejando a los agentes humanos libres para manejar cuestiones más complejas. Esto